In [6]:
import pandas as pd
import numpy as np
import re

In [7]:
txt_file = open("dietGMTKN.txt", "r")

Molecule_Set = txt_file.readlines()
for i in range(len(Molecule_Set)):
    Molecule_Set[i] = Molecule_Set[i].rstrip()

In [17]:
method = 'M06-2X'
basis = 'pcseg-0'


DataFile = pd.read_csv(f'29Sept/Results/Energies_{method}.{basis}.txt', sep = ' ')
#Reads original datafile (NOT CREATED BY SAM)
DataFile

,"Molecule,",Energy_Hartree
0,ACONF_H_g+x-t+,-236.364838
1,ACONF_H_ttt,-236.367251
2,ACONF_H_x+g-g-,-236.365202
3,AHB21_10,-151.695399
4,AHB21_10A,-75.454750
...,...,...
328,YBDE18_nh3,-56.350891
329,YBDE18_pf3-cbh22,-729.411245
330,YBDE18_pf3,-639.499943
331,YBDE18_ph3-ch2,-381.742863


In [18]:
New_Molecule_Frame = pd.DataFrame(columns = ['Molecule', 'Value'])
for Molecule in Molecule_Set:
    
    try:
    #print(Molecule)
        with open(f'output_dir/{method}/{basis}/{Molecule}/mol.out', 'r') as f:
    # Reads file produced by Sam :D

            for line in f.readlines():
            #Checks for this phrase in each line
                if 'Total energy in the final basis set = ' in line:
                #Checks for -, ., numbers etc.
                    numbers = re.compile(r"-.\d+\.\d*")
                    result = list(map(float, numbers.findall(line)))
                
                #Converts from list to floats
                    Molecule_Value = float(result[0])
                
                #Creates the New DataFrame
                    New_Molecule_Frame.loc[Molecule] = [Molecule,Molecule_Value]
                    New_Molecule_Frame.reset_index()
                
                #Swaps values out
                    for i in range(len(New_Molecule_Frame)):

                        for j in range(len(DataFile)):

                            if DataFile.iloc[j,0] == New_Molecule_Frame.iloc[i,0]:
                            
                                DataFile.iloc[j,1] = New_Molecule_Frame.iloc[i,1]

    except: 
        continue
f.close()
DataFile

,"Molecule,",Energy_Hartree
0,ACONF_H_g+x-t+,-236.364838
1,ACONF_H_ttt,-236.367251
2,ACONF_H_x+g-g-,-236.365202
3,AHB21_10,-151.695399
4,AHB21_10A,-75.454750
...,...,...
328,YBDE18_nh3,-56.350891
329,YBDE18_pf3-cbh22,-729.411245
330,YBDE18_pf3,-639.499943
331,YBDE18_ph3-ch2,-381.742863


In [19]:
DataFile.to_csv(path_or_buf=f'Correction_Data/Corrected_Data_{method}.{basis}.csv', sep=',', index = False)